In [ ]:
%%bash
mkdir -p /kaggle/working/submission
pip install bitsandbytes accelerate
pip install -U transformers

**Keep Secrets:** Add-ons>Secrets

In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None

try:
    HF_TOKEN = secrets.get_secret("hf_token")
except:
    pass

**Download the LLM Locally.** gemma2 could not be executed by calling it from input. so downloaded it locally.

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/working/submission/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)
g_model_path.mkdir(parents=True)

snapshot_download(
    repo_id="google/gemma-2-9b-it",
    ignore_patterns="original*",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=globals().get("HF_TOKEN", None)
)

In [ ]:
!ls -l /kaggle/working/submission/model

**Device Mapping for Model Layers**
When dealing with large models, distributing the layers across multiple devices can help in managing memory and computational resources more efficiently. The device_maps variable is used to specify which layers of the model are assigned to which device.

By using torch.backends.cuda.enable_mem_efficient_sdp(False), you can disable memory-efficient SDP in PyTorch, which might be necessary for certain models or debugging purposes. This configuration can help achieve better performance or consistency across different hardware setups.

[How to prompt Gemma 2](https://huggingface.co/blog/gemma2#how-to-prompt-gemma-2)

In [ ]:
%%writefile -a submission/main.py

import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

torch.backends.cuda.enable_mem_efficient_sdp(False)

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "model")
else:
    model_id = "/kaggle/working/submission/model"

device_maps = [('model.layers.0', 0),
 ('model.layers.1', 0),
 ('model.layers.2', 0),
 ('model.layers.3', 0),
 ('model.layers.4', 0),
 ('model.layers.5', 0),
 ('model.layers.6', 0),
 ('model.layers.7', 0),
 ('model.layers.8', 0),
 ('model.layers.9', 0),
 ('model.layers.10', 0),
 ('model.layers.11', 0),
 ('model.layers.12', 0),
 ('model.layers.13', 0),
 ('model.layers.14', 0),
 ('model.layers.15', 0),
 ('model.layers.16', 0),
 ('model.layers.17', 0),
 ('model.layers.18', 0),
 ('model.layers.19', 1),
 ('model.layers.20', 1),
 ('model.layers.21', 1),
 ('model.layers.22', 1),
 ('model.layers.23', 1),
 ('model.layers.24', 1),
 ('model.layers.25', 1),
 ('model.layers.26', 1),
 ('model.layers.27', 1),
 ('model.layers.28', 1),
 ('model.layers.29', 1),
 ('model.layers.30', 1),
 ('model.layers.31', 1),
 ('model.layers.32', 1),
 ('model.layers.33', 1),
 ('model.layers.34', 1),
 ('model.layers.35', 1),
 ('model.layers.36', 1),
 ('model.layers.37', 1),
 ('model.layers.38', 1),
 ('model.layers.39', 1),
 ('model.layers.40', 1),
 ('model.layers.41', 1),
 ('model.embed_tokens', 1),
 ('model.layers', 1)]

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id) 
id_eot = tokenizer.convert_tokens_to_ids(["<eos>"])[0]

device = {layer:gpu_mem for (layer,gpu_mem) in device_maps}
config = AutoConfig.from_pretrained(model_id)
config.gradient_checkpointing = True
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", quantization_config=quantization_config,
                                             device_map="auto", trust_remote_code=True, config=config)

def generate_answer(template):
    input_ids = tokenizer(template, return_tensors="pt").to("cuda")
    output_ids = model.generate(**input_ids, max_new_tokens=15).squeeze()
    start_gen = input_ids.input_ids.shape[1]
    output_ids = output_ids[start_gen:]
    if id_eot in output_ids:
        stop = output_ids.tolist().index(id_eot)
        output = tokenizer.decode(output_ids[:stop])
    else:
        output = tokenizer.decode(output_ids)
    output = re.sub('\n', '', output)
    output = re.sub(' <end_of_turn>', '', output)
    output = re.sub('<end_of_turn>', '', output)
    return output

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            output = self.asker(obs)
        if mode == "answering":
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"
            else:
                output = "yes"
        if mode == "guessing":
            output = self.asker(obs)
        return output

    def asker(self, obs):
        sys_prompt = """
        You are an AI assistant designed to play the 20 Questions game. 
        In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner.
        The keyword is a specific "thing" or "place".
        """
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            Let's play 20 Questions. You are playing the role of the Questioner.
            Please ask a yes-or-no question.
            to help you, here's an example of how it should work assuming that the keyword is Morocco:
            examle:
            <you: is it a contry?
            user: yes
            you: is it in europe?
            user: no
            you: is it in africa?
            user: yes
            you: do most people living there have dark skin?
            user: no
            you: is it a country name starting by m ?
            user: yes
            you: is it Morocco?
            user: yes.>
            the user has chosen the word, ask your first question!
            please be short and not verbose, give only one question, no extra word!
            """
        
            chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
            chat_template += "<start_of_turn>model\n"

            if len(obs.questions)>=1:
                    for q, a in zip(obs.questions, obs.answers):
                        chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                        chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
                    
        elif obs.turnType == "guess":
                conv = ""
                for q, a in zip(obs.questions, obs.answers):
                    conv += f"""Question: {q}\nAnswer: {a}\n"""
                guess_prompt =  sys_prompt + f"""
                so far, the current state of the game is as following:\n{conv}
                based on the conversation, can you guess the word, please give only the word, no verbosity around
                """
                chat_template = f"""<start_of_turn>system\n{guess_prompt}<end_of_turn>\n"""
                chat_template += "<start_of_turn>model\n"

        output = generate_answer(chat_template)        
        return output
    
    def answerer(self, obs):
        ask_prompt = f"""
        You are an AI assistant designed to play the 20 Questions game. 
        In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner.
        The keyword is a specific place, or thing.\n
        so make sure you understand the user's question and you understand the keyword you're playig on.
        for now the word that the user should guess is: "{obs.keyword}", it is of category "{obs.category}",
        to help you, here's an example of how it should work assuming that the keyword is Morocco in the category "place":
        examle:
        <user: is it a place?
        you: yes
        user: is it in europe?
        you: no
        user: is it in africa?
        you: yes
        user: do most people living there have dark skin?
        you: no
        user: is it a country name starting by m ?
        you: yes
        user: is it Morocco?
        you: yes.>
        """
        chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""
        chat_template += "<start_of_turn>user\n"
        chat_template += f"{obs.questions[0]}"
        chat_template += "<start_of_turn>model\n"
        if len(obs.answers)>=1:
            for q, a in zip(obs.questions[1:], obs.answers):
                chat_template += f"{q}<end_of_turn>\n<start_of_turn>user\n"
                chat_template += f"{a}<end_of_turn>\n<start_of_turn>model\n"
        output = generate_answer(chat_template)
        return output

robot = Robot()

def agent(obs, cfg):
    
    if obs.turnType =="ask":
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:
        response = "yes"
        
    return response

In [ ]:
# def simple_agent1(obs, cfg):
#     # if agent is guesser and turnType is "ask"
#     if obs.turnType == "ask":
#         response_list = [
#             'Is it in Africa?',
#             'Is it in America?',
#             'Is it in Asia?',
#             'Is it in Oceania?',
#             'Is it in Eastern Europe?',
#             'Is it in Northern Europe?',
#             'Is it in Southern Europe?',
#             'Is it in Western Europe?',
#             'Is it Japan?'
#         ]
# #         response = response_list[len(obs.questions)]
#         response = random.choice(response_list)
#     elif obs.turnType == "guess":
#         response = "duck"
#     elif obs.turnType == "answer":
#         response = random.choices(["yes", "no"])[0]
#     return response

In [ ]:
# %%time

# import random
# from kaggle_environments import make
# agent = "/kaggle/working/submission/main.py"
# env = make("llm_20_questions", debug=True)
# game_output = env.run(agents=[agent, simple_agent1, simple_agent1, simple_agent1])

In [ ]:
# env.render(mode="ipython", width=600, height=500)

In [ ]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

class Observation:
    def __init__(self):
        self.step = 0
        self.role = "guesser"
        self.turnType = "ask"
        self.keyword = "Japan"
        self.category = "country"
        self.questions = []
        self.answers = []
        self.guesses = []
        
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)

    keyword_list = []
    category_list = []
    alts_list = []

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)
            category_list.append(json_data[i]['category'])
            alts_list.append(json_data[i]['words'][j]['alts'])

    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])
    data_pd['keyword'] = keyword_list
    data_pd['category'] = category_list
    data_pd['alts'] = alts_list
    
    return data_pd
    
keywords_df = create_keyword_df(KEYWORDS_JSON)

In [ ]:
obs = Observation()
cfg = "_"

sample_df = keywords_df.sample()
obs.keyword = sample_df["keyword"].values[0]
obs.category = sample_df["category"].values[0]
alts_list = sample_df["alts"].values[0]
alts_list.append(obs.keyword)

print(f"keyword:{obs.keyword}")

for round in range(20):
    obs.step = round+1
    
    obs.role = "guesser"
    obs.turnType = "ask"
    question = agent(obs, cfg)
    obs.questions.append(question)
    
    obs.role = "answerer"
    obs.turnType = "answer"
    answer = agent(obs, cfg)
    obs.answers.append(answer)
    
    obs.role = "guesser"
    obs.turnType = "guess"
    guess = agent(obs, cfg)
    obs.guesses.append(guess)
    
    print(f"round: {round+1}")
    print(f"question: {question}")
    print(f"answer: {answer}")
    print(f"guess: {guess}")
    
    if guess in alts_list:
        print("Win!!")
        break

**I could not submit...** The tar file could not be created because the output directory exceeded its capacity.

In [ ]:
# !apt install pigz pv > /dev/null

In [ ]:
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

In [ ]:
# # to see what's inside tar.gz file

# import tarfile
# tar = tarfile.open("/kaggle/working/submission.tar.gz")
# for file in tar.getmembers():
#     print(file.name)